In [1]:
from genericpath import isfile
import os
import re
import pickle
import logging
import netCDF4
from calendar import monthrange
from tqdm import tqdm
from glob import glob
from typing import Iterable
from datetime import datetime, timedelta, date
import numpy as np
import pandas as pd
import xarray as xr
# import xesmf as xe
from argparse import ArgumentParser
import h5py
import sys
sys.path.insert(1, "../")

# logger = logging.getLogger(__file__) this is for python scripts
##🚩 update this for jupyter notebook
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


In [ ]:

#run this first in the terminal to authenticate with GCP
# gcloud auth application-default login 


In [ ]:
!pip install tensorflow

In [2]:
import os

# Only needed for tensorflow versions greater than 2.16.0
os.environ["TF_USE_LEGACY_KERAS"] = "1"

import sys
sys.path.insert(1,"../scripts/")
# import forecast_gfs
# import forecast/

In [5]:
import gcsfs
import pickle
import xarray as xr

# Automatic connection using default credentials
fs = gcsfs.GCSFileSystem()

# Read a NetCDF file from Google Cloud Storage
with fs.open('gs://melvin_aims_bucket/2024/20240101_00.nc') as f_obj:
    ds = xr.open_dataset(f_obj, engine='h5netcdf')


/home/melvin/miniconda3/envs/cgan_new/lib/python3.11/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [6]:
ds

<xarray.Dataset> Size: 544kB
Dimensions:        (latitude: 384, longitude: 352, time: 1)
Coordinates:
  * latitude       (latitude) float32 2kB -13.65 -13.55 -13.45 ... 24.55 24.65
  * longitude      (longitude) float32 1kB 19.15 19.25 19.35 ... 54.15 54.25
  * time           (time) datetime64[ns] 8B 2024-01-01
Data variables:
    precipitation  (latitude, longitude) float32 541kB ...
Attributes:
    description:  6h average of IMERG V07 rainfall data in the ICPAC region

In [ ]:
import importlib
importlib.reload(forecast_gfs)

# forecast_gfs.make_fcst()

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
dates = np.asarray(fcst_params["INPUT"]["dates"],dtype='datetime64[D]')
d = dates[0].astype(object)
path_to_file = fcst_params["OUTPUT"]["folder"]+f"test/{d.year}/GAN_{d.year}{d.month:02}{d.day:02}.nc"

ds = xr.open_dataset(path_to_file)


fig = plt.figure(figsize=(14,8))
ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()
ax.add_feature(cfeature.BORDERS, linewidth=1)
c = plt.pcolormesh(ds.longitude.values, ds.latitude.values, np.squeeze(ds.precipitation.mean(['valid_time','member']).values*24),
                   transform=ccrs.PlateCarree(), cmap='YlGnBu',vmin=1,vmax=60)

cb = plt.colorbar(c, fraction=0.02)